In [2]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv


# Loading vairable from .env
load_dotenv()

#Getting them on this file

API_ID = os.getenv("ADZUNA_APP_ID")
API_KEY = os.getenv("ADZUNA_APP_KEY")

bfd76e0a


In [4]:
url = f"https://api.adzuna.com/v1/api/jobs/gb/search/1?app_id={API_ID}&app_key={API_KEY}&results_per_page=20&what=javascript%20developer&content-type=application/json"

response = requests.get(url)
data = response.json()
print(data)


ProxyError: HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: http://api.adzuna.com/v1/api/jobs/gb/search/1?app_id=bfd76e0a&app_key=119570688754f7edc674415679c5e689&results_per_page=20&what=javascript%20developer&content-type=application/json (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPConnection object at 0x139c57f50>: Failed to establish a new connection: [Errno 61] Connection refused')))